<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Análisis de sentimiento
`Fecha de entrega: Noviembre 21, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
# cargar librerias 

# Estos paquetes los usaremos para analizar y visualizar los resultados
import pandas as pd
import tweepy
import numpy as np
import re
import regex
import emoji

# Libreias Plotly
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)
import seaborn as sns

from collections import Counter

import urllib.parse
import urllib.request
import time
import hmac
import hashlib
import random
import base64
import json

# Este es un modulo local de Python que contiene las credenciales de acceso para Twitter
import tweepy

from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

from flair.models import TextClassifier
from flair.data import Sentence

from wordcloud import WordCloud
from wordcloud import ImageColorGenerator

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')
import PIL.Image
from IPython.display import display

C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning:

CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)



###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

### Repetimos el ejercio del taller anterior "taller10" parta obtener los datos de twiter 

In [2]:
# Leer llaves
keys = pd.read_csv("C:/Users/practicas/Documents/Claves Tuit.csv", header=None)
keys= dict(zip(keys[0],keys[1]))

In [3]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'],keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True) # se autoregula para recolectarinformacion

In [4]:
# Cargar archivo ciudades lat.-ln. bogota
ciudades = pd.read_csv("C:/Users/practicas/Documents/co_small.csv")
ciudades.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Bogotá,4.649178,-74.062827,Colombia,CO,Bogotá,primary,7772000.0,6333661.0
1,Medellín,6.251840,-75.563591,Colombia,CO,Antioquia,admin,3297000.0,1999979.0
2,Cali,3.437222,-76.522500,Colombia,CO,Valle del Cauca,admin,2254000.0,2178836.0
3,Barranquilla,10.963889,-74.796389,Colombia,CO,Atlántico,admin,1798000.0,1244491.0
4,Bucaramanga,7.125393,-73.119804,Colombia,CO,Santander,admin,1009000.0,571820.0


In [5]:
tweets = tweepy.Cursor(api.search,
                       q=['Selección Colombia futbol -RT'], # RT no incluir los retuis
                       lang='es', # solo tui en español
                           geocode=f'4.649178,-74.062827,45mi', #direccion geografica  bogota cambia a la que se quiera
                       tweet_mode='extended', # para que no recorte los texto del tuit
                       include_rts=False).items(100) # limitar el numero de twitwes

In [6]:
t = []
for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    
    t.append(temp)

In [7]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date']
data.head()

,tweet,date
0,El mensaje de James para la Selección y el paí...,2020-11-16 03:04:34
1,El mal trabajador le echa la culpa a las herra...,2020-11-16 02:09:48
2,"Noviembre 13 - Bogotá, día de fútbol, un part...",2020-11-16 00:17:33
3,Por favor sr Queiroz....asuma que a la selecci...,2020-11-15 23:32:03
4,La crítica de Queiroz al árbitro de Colombia-U...,2020-11-15 20:39:57


In [8]:
#quitar enlaces y menciones
#quitar emojis

import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [9]:
#traducir tweet a ingles

translator = Translator()
data['ing'] = data['tweet'].apply(lambda valor: translator.translate(valor, src='es', dest='en'))
data.head()

,tweet,date,ing
0,El mensaje de James para la Selección y el paí...,2020-11-16 03:04:34,"Translated(src=es, dest=en, text=James's messa..."
1,El mal trabajador le echa la culpa a las herra...,2020-11-16 02:09:48,"Translated(src=es, dest=en, text=The bad worke..."
2,"Noviembre 13 - Bogotá, día de fútbol, un part...",2020-11-16 00:17:33,"Translated(src=es, dest=en, text=November 13 -..."
3,Por favor sr Queiroz....asuma que a la selecci...,2020-11-15 23:32:03,"Translated(src=es, dest=en, text=Please, Mr. Q..."
4,La crítica de Queiroz al árbitro de Colombia-U...,2020-11-15 20:39:57,"Translated(src=es, dest=en, text=Queiroz's cri..."


## Librerias  para analisis de sentimientos vistas en clases
#### no es nesesario lipiar la data

In [11]:
# Modelos BvaderSentiment

#data['Sentiment'] = data['ingles'].apply(lambda valor:vader.polarity_scores(valor)['compound'])
#data.head()

In [13]:
#Modelos Basados en Redes Neuronales 
classifier = TextClassifier.load('en-sentiment')

def sent_flair(texto):
    sentence = Sentence(texto)
    classifier.predict(sentence)
    result = sentence.labels[0]
    label = result.value
    score = result.score
    if label == 'POSITIVE':
        return score
    if label == 'NEGATIVE':
        return -1 * score
    return score

data['sentimiento'] = data['ingles'].apply(lambda valor: sent_flair(valor))
data.head()

2020-11-15 23:06:59,629 loading file C:\Users\practicas\.flair\models\sentiment-en-mix-distillbert_3.1.pt


,tweet,date,ing,ingles,sentimiento
0,El mensaje de James para la Selección y el paí...,2020-11-16 03:04:34,"Translated(src=es, dest=en, text=James's messa...",James's message for the National Team and the ...,0.995395
1,El mal trabajador le echa la culpa a las herra...,2020-11-16 02:09:48,"Translated(src=es, dest=en, text=The bad worke...",The bad worker blames the tools\nhttps://t.co/...,-0.999821
2,"Noviembre 13 - Bogotá, día de fútbol, un part...",2020-11-16 00:17:33,"Translated(src=es, dest=en, text=November 13 -...","November 13 - Bogotá, football day, a match ne...",-0.934919
3,Por favor sr Queiroz....asuma que a la selecci...,2020-11-15 23:32:03,"Translated(src=es, dest=en, text=Please, Mr. Q...","Please, Mr. Queiroz .... assume that the natio...",-0.999260
4,La crítica de Queiroz al árbitro de Colombia-U...,2020-11-15 20:39:57,"Translated(src=es, dest=en, text=Queiroz's cri...",Queiroz's criticism of the Colombian-Uruguay r...,-0.997736


## Columna con el puntaje numérico del sentimiento

In [14]:
data[['tweet', 'sentimiento']].sort_values('sentimiento')
data.head()

,tweet,date,ing,ingles,sentimiento
0,El mensaje de James para la Selección y el paí...,2020-11-16 03:04:34,"Translated(src=es, dest=en, text=James's messa...",James's message for the National Team and the ...,0.995395
1,El mal trabajador le echa la culpa a las herra...,2020-11-16 02:09:48,"Translated(src=es, dest=en, text=The bad worke...",The bad worker blames the tools\nhttps://t.co/...,-0.999821
2,"Noviembre 13 - Bogotá, día de fútbol, un part...",2020-11-16 00:17:33,"Translated(src=es, dest=en, text=November 13 -...","November 13 - Bogotá, football day, a match ne...",-0.934919
3,Por favor sr Queiroz....asuma que a la selecci...,2020-11-15 23:32:03,"Translated(src=es, dest=en, text=Please, Mr. Q...","Please, Mr. Queiroz .... assume that the natio...",-0.999260
4,La crítica de Queiroz al árbitro de Colombia-U...,2020-11-15 20:39:57,"Translated(src=es, dest=en, text=Queiroz's cri...",Queiroz's criticism of the Colombian-Uruguay r...,-0.997736


In [15]:
data.tail()

,tweet,date,ing,ingles,sentimiento
95,"La Selección Colombia para mi no son penas, lo...",2020-11-13 21:51:17,"Translated(src=es, dest=en, text=The Colombian...",The Colombian National Team for me are not pen...,0.997195
96,#NuestraSele | #COL 0-2 #URU | ⏱ 54' | ¡GOL! d...,2020-11-13 21:48:07,"Translated(src=es, dest=en, text=#OurSele | #C...",#OurSele | #COL 0-2 #URU | ⏱ 54 '| GOAL! from ...,0.986182
97,#NuestraSele | #COL 0-1 #URU |⏱ 51' | Penal a ...,2020-11-13 21:45:44,"Translated(src=es, dest=en, text=#OurSele | #C...",#OurSele | #COL 0-1 #URU | ⏱ 51 '| Penalty in ...,-0.999769
98,Así fue el gol del atacante uruguayo Edinson C...,2020-11-13 21:45:38,"Translated(src=es, dest=en, text=This was the ...",This was the goal of the Uruguayan attacker Ed...,0.961208
99,#NuestraSele | #COL 0-1 #URU |⏱ 49' | ¡PAAAALO...,2020-11-13 21:43:49,"Translated(src=es, dest=en, text=#OurSele | #C...",#OurSele | #COL 0-1 #URU | ⏱ 49 '| PAAAALO! Ur...,-0.936629


### [10 pts] Punto 2: ¿Cuál es el tuit más positivo y más negativo?

In [20]:
#tuit Positivo
data.loc[0]['tweet']

'El mensaje de James para la Selección y el país ► https://t.co/U1KfGGEkvJ https://t.co/pic5yQ4Xal'

In [18]:
#tuit Negativo
data.loc[99]['tweet']

"#NuestraSele | #COL 0-1 #URU |⏱ 49' | ¡PAAAALO! Se salva Uruguay \n\n¡Vamos, Colombia! 🇨🇴🇨🇴\n#EliminatoriasSudamericanas \n\nSigue aquí el #MinutoAMinuto: https://t.co/TKMqnsG20x"

### `[15 pts]` Punto 3: ¿Cuántos tuits positivos/negativos hay en sus datos?

from sent_flair.sent_flair import SentimentIntensityAnalyzer
 
analyzer = SentimentIntensityAnalyzer()
 
pos_count = 0
pos_correct = 0
 
with open("positive.txt","r") as f:
for line in f.read().split('\n'):
    vs = analyzer.polarity_scores(line)
    if vs['compound'] > 0:
    pos_correct += 1
    pos_count +=1
 
    neg_count = 0
    neg_correct = 0

with open("negative.txt","r") as f:
for line in f.read().split('\n'):
    vs = analyzer.polarity_scores(line)
    if vs['compound'] <= 0:
    neg_correct += 1
    neg_count +=1
    
print("Positiva = {}% via {} samples".format(pos_correct/pos_count*100.0, pos_count))
print("Precisión Negativa = {}% via {} samples".format(neg_correct/neg_count*100.0, neg_count))

###  `[BONUS 5pts]` : Diagrama de barras de la pregunta 3

###  `[BONUS 7pts]` : Nubes de palabras por sentimiento